# Part I: Introductory Material

## Chapter 2: The Basics

In [ ]:
int i1 = 1;
int i2{1};
auto i3{1.2};

cout << i1 << ' ' << i2 << ' ' << i3 << endl;

In [ ]:
double sum(const vector<double>&v) { 
    double s = 0;
    for (double x : v) {
        s += x;
    }
    return s;
}

In [ ]:
constexpr double square(double x) { return x * x; }

 `const` : value can not be changed.
 

 `constexpr`: to be valued at compile time.

In [ ]:
const int a = 1;
int b = 2;
constexpr double c1 = 1.4 * a;
// constexpr double c2 = 1.4 * b; // error

vector<double> v {1.2, 3.4, 4.5};
const double s1 = sum(v);
cout << s1 << endl;

// constexpr double s2 = sum(v); // error
constexpr double s3 = square(a);


`*`:
* in declaration: pointer.
* otherwise: contents of.

`&`: 
* in declaration: reference to.
* otherwise: address of.

`.`: access `struct` members through a name or reference.

`->`: access `sturct` members through a pointer.


In [ ]:
enum class Color { red, blue, green };

Color col = Color::red;


In [ ]:
namespace MyCode {
    class complex { };
    complex sqrt(complex);
    int main();
}

In [ ]:
static_assert(4 <= sizeof(int), "intergers are too small"); // check at compile time

## Chapter 3: Abstraction Mechanisms

`std::initializer_list`: define the initializer-list constructor

In [ ]:
Vector::Vector<std:initializer_list<double> lst):elem{new double[lst.size()], sz{lst.size()}} {
    copy(lst.begin(), lst.end(), elem);
}

In [ ]:
class Point
{
    private:
        int x, y;
    public:
        Point() : x(0), y(0) {}
        Point(int x, int y) : x(x), y(y) {}
}

In [ ]:
class Shape {
public:
    virtual Point center() const=0;
    virtual void move(Point to)=0;
    
    virtual void draw() const=0;
    virtual void rotate(int angle)=0;
    
    virtual ~Shape(){}
    
};

In [ ]:
void rotate_all(vector<Shape*>& v, int angle) 
{
    for (auto p : v)
        p->rotate(angle);
}

In [ ]:
class Circle : public Shape {
public:
    Circle(Point p, int rr);
    
    Point center() const { return x; }
    void move(Point to) { x = to; }
    
    void draw() const;
    void rotate(int) {}
    
private:
    Point x;
    int r;
};

In [ ]:
class Smiley : public Circle {
public:
    Smiley(Point p, int r): Circle{p, r}, mouth{ nullptr } {}
    
    ~Smiley()
    {
        delete mouth;
        for (auto p : eyes) delete p;
    }
    
    void move(Point to);
    
    void draw() const;
    void rotate(int);
    
    void add_eye(Shape* s) { eyes.push_back(s); }
    void set_mouth(Shape* s);
    virtual void wink(int i);
    
private:
    vector<Shape*> eyes;
    Shape* mouth;
}

void Smiley::draw()
{
    Circle::draw();
    for (auto p : eyes)
        p->draw();
    mouth->draw();
}


* copy constructor: called when class was created.
* copy assignment：called when assigned. 

In [ ]:
class A {
 public:
  A() {
    std::cout << "A() called!\n";
  }
  A(const A& a) {
    std::cout << "A(const A& a) called!\n";
  }
  A& operator=(const A& a) {
    std::cout << "operator=(const A& a)!\n";
    return *this;
  }
};


In [ ]:
 A a1;
 A a2 = a1;
 A a3(a1);
 a3 = a2;

In [ ]:
// function objects
template<typename T>
class LessTan {
    const T val;
public:
    LessTan(const T& v):val(v) {}
    bool operator()(const T& x) const { return x < val; }
};

In [ ]:
LessTan<int> lti{42};
cout << lti(10);

##  Chapter 4: Containers and Algorithms

In [ ]:
// I/O
struct Entry {
    string name;
    int number;
};


In [ ]:
int sum(){ return 1;}

In [ ]:
ostream& operator<<(ostream& os, const Entry& e) 
{
    return os << e.name << ' ' << e.number << endl;
}

In [2]:
#include <iostream>
#include <algorithm>
#include <vector>
using namespace std;

* `unique_copy`

In [ ]:
// back_insert, unique_copy
vector<int>a{1, 2, 3, 2, 3, 4, 5};
sort(a.begin(), a.end());
vector<int>b(10);
unique_copy(a.begin(), a.end(), b.begin());


* `find`

In [3]:
vector<int>a{1, 2, 3, 4, 5};
auto p = find(a.begin(), a.end(), 2);
cout << p - a.begin() << endl;

1


In [5]:
template<typename C, typename V>
vector<typename C::iterator> find_all(C& c, V v) {
    vector<typename C::iterator> res;
    for (auto p = c.begin(); p != c.end(); ++p) {
        if (*p == v)
            res.push_back(p);
    }
    return res;
}
auto ps = find_all(a, 2);

In [8]:
template<typename T>
using Iterator = typename T::iterator;

template<typename C, typename V>
vector<Iterator<C>> find_all(C& c, V v) {
    vector<Iterator<C>> res;
    for (auto p = c.begin(); p != c.end(); ++p) {
        if (*p == v)
            res.push_back(p);
    }
    return res;
}
auto ps = find_all(a, 2);

* `unique_ptr`: to represent unique ownership
* `shared_ptr`: to represent shared ownership 

## Chapter 5: Concurrency and Utilities

* `join`: wait for the thread to terminate. 

In [ ]:
mutex m; // controlling mutex
int sh; // shared data
void f()
{
    unique_lock<mutex> lck {m}; // acquire mutex
    sh += 7; // manipulate shared data
}

In [ ]:
// avoid dead lock
void f() {
    unique_lock<mutex> lck1 {m1, defer_lock};
    unique_lock<mutex> lck2 {m2, defer_lock};
    unique_lock<mutex> lck3 {m3, defer_lock};
    
    lock(lck1, lck2, lck3);
}

In [ ]:
class Message {
    
};
queue<Message> mqueue;
condition_variable mcond;
mutex mmutex;

void consumer() {
    while (true) {
        unique_lock<mutex> lck{mmutex};
        while (mcond.wait(lck));
        auto m = mqueue.front();
        mqueue.pop();
        lck.unlock();
    }
}

void producer() {
    while (true) {
        Message m;
        unique_lock<mutex> lck {mmutex};
        mqueue.push(m);
        mcond.notify_one();
    }
}

* `future`, `promise`: return a value from a task spawned on a separated thread 
* `packaged_task`: help launch tasks and connect up the mechanisms for returning a result
* `async`: launch of a task in a manner very similar to calling a function

In [ ]:
void f(promise<X>& px) // a task: place the result in px
{
    // ...
    try {
        X res;
        // ... compute a value for res ...
        px.set_value(res);
    }
    catch (...) { // oops: couldn’t compute res
        // pass the exception to the future’s thread:
        px.set_exception(current_exception());
    }
}
void g(future<X>& fx) // a task: get the result from fx
{
    // ...
    try {
        X v = fx.g et(); // if necessary, wait for the value to get computed
        // ... use v ...
        }
    catch (...) { // oops: someone couldn’t compute v
        // ... handle error ...
    }
}

In [ ]:
double accum(double∗ beg, double ∗ end, double init)
// compute the sum of [beg:end) starting with the initial value init
{
    return accumulate(beg,end,init);
}

double comp2(vector<double>& v)
{
    using Task_type = double(double∗,double∗,double); // type of task
    
    packaged_task<Task_type> pt0 {accum}; // package the task (i.e., accum)
    packaged_task<Task_type> pt1 {accum};
    
    future<double> f0 {pt0.get_future()}; // get hold of pt0’s future
    future<double> f1 {pt1.get_future()}; // get hold of pt1’s future
    
    double∗ first = &v[0];
    thread t1 {move(pt0),first,first+v.siz e()/2,0}; // star t a thread for pt0
    thread t2 {move(pt1),first+v.siz e()/2,first+v.siz e(),0}; // star t a thread for pt1
    // ...
    return f0.get() + f1.get();
}

In [ ]:
double comp4(vector<double>& v)
// spawn many tasks if v is large enough
{
    if (v.siz e()<10000) return accum(v.begin(),v.end(),0.0);
    auto v0 = &v[0];
    auto sz = v.siz e();
    auto f0 = async(accum,v0,v0+sz/4,0.0); // first quarter
    auto f1 = async(accum,v0+sz/4,v0+sz/2,0.0); // second quarter
    auto f2 = async(accum,v0+sz/2,v0+sz∗3/4,0.0); // third quarter
    auto f3 = async(accum,v0+sz∗3/4,v0+sz,0.0); // four th quar ter
    return f0.get()+f1.g et()+f2.get()+f3.get(); // collect and combine the results
}   

In [ ]:
* `decltype()`: returns the declared type of its argument. 

In [ ]:
void f(valarray<double>& a1, valarray<double>& a2) {
    valarray<double> a = a1 * 3.14 + a2 / a1;
    a2 += a1 * 3.14;
    a = abs(a);
    double d = a2[7];
}

In [9]:
#include <limits>

In [11]:
cout << numeric_limits<int>::max();

2147483647

# Part II: Basic Facilities

## Chapter 6: Types and Declarations


In [2]:
bool b1{ 1 == 2 };

// non-null pointer converts to true,
// pointers with value nullptr converts to false.
int *p;
bool b = p;
bool b2 { p != nullptr };

* `wchar_t`: Hold a larger character set, such as Unicode
* `char16_t`: Hold 16-bit character set, such as UTF-16 
* `char32_t`: Hold 32-bit character set, such as UTF-32 

* Octal: '\60', 060
* Hexadecimal: '\x30', 0x30
* Decimal: 48

In [10]:
#include<iostream>
auto ac = alignof('c');

input_line_22:2:12: warning: 'alignof' applied to an expression is a GNU extension [-Wgnu-alignof-expression]
 auto ac = alignof('c');
           ^
input_line_22:3:14: error: use of undeclared identifier 'a'
alignas(int) a;
             ^


Interpreter Error: 

In [ ]:
double sqrt(double); // function declaration
extern int error_number; // variable declaration
struct User; // type name declaration

In [ ]:
char* kings[]; // array of pointers to char
char(*kings)[]; // pointer to an arrar of char    

In [ ]:
// prefer = when using auto.
auto z1 {99}; // z1 is an initializer_list<int>
auto z2 = 99; // z2 is an int

vector<int> v1 {99}; // v1 is a vector of 1 element with the value 99
vector<int> v2(99); // v2 is a vector of 99 elements each with the default value 0

char buf[max] {}; // initialize every char to 0

In [ ]:
template<class T, class U>
auto operator+(const Matrix<T>& a, const Matrix<U>& b) −> Matrix<decltype(T{}+U{})>
{
    Matrix<decltype(T{}+U{})> res;
    for (int i=0; i!=a.rows(); ++i)
        for (int j=0; j!=a.cols(); ++j)
            res(i,j) += a(i,j) + b(i,j);
    
    return res;
}

In [ ]:
# Type Aliases
using Pchar = char∗; // pointer to character
using PF = int(∗)(double); // pointer to function taking a double and returning an int

# Chapter 7: Pointers, Arrays, and References

In [6]:
#include<iostream>
// raw string
R"("quoted string")";
char a[] = "1234";
std::cout << a[2] << ' ' << 2[a];

3 3

In [ ]:
void f1(char∗ p)
{
    char s[] = "Gorm";
    const char∗ pc = s; // pointer to constant
    pc[3] = 'g'; // error : pc points to constant
    pc = p; // OK
    
    char ∗const cp = s; // constant pointer
    cp[3] = 'a'; // OK
    cp = p; // error : cp is constant
    
    const char ∗const cpc = s; // const pointer to const
    cpc[3] = 'a'; // error : cpc points to constant
    cpc = p; // error : cpc is constant
}

char ∗const cp; // const pointer to char
char const∗ pc; // pointer to const char
const char∗ pc2; // pointer to const char

* lvalue references: to refer to objects whose value we want to change
* const references: to refer to objects whose value we do not want to change (e.g., a constant)
* rvalue references: to refer to objects whose value we do not need to preserve after we have used it (e.g., a temporary)

In [10]:
int var = 0;
int& rr {var};
++rr;
std::cout<<rr;

const double& dr = 1;

1

In [ ]:
string var {"Cambridge"};
string f();
string& r1 {var}; // lvalue reference, bind r1 to var (an lvalue)
string& r2 {f()}; // lvalue reference, error : f() is an rvalue
string& r3 {"Princeton"}; // lvalue reference, error : cannot bind to temporar y

string&& rr1 {f()}; // rvalue reference, fine: bind rr1 to rvalue (a temporar y)
string&& rr2 {var}; // rvalue reference, error : var is an lvalue
string&& rr3 {"Oxford"}; // rr3 refers to a temporar y holding "Oxford"
const string cr1& {"Harvard"}; // OK: make temporar y and bind to cr1

# Chapter 8: Structures, Unions, and Enumerations

In [ ]:
//  the compiler is not able to determine the size of No_good.
struct No_good {
    No_good member; // error : recursive definition
};

# Chapter 9: Statements

In [ ]:
for (string s; cin>>s;)
    v.push_back(s);

In [ ]:
void do_something(int i, int j)
// do something to a two-dimensional matrix called mn
{
    for (i = 0; i!=n; ++i)
        for (j = 0; j!=m; ++j)
            if (nm[i][j] == a)
                goto found;
    
        // not found
        // ...
    found:
    // nm[i][j] == a
}

# Chapter 10: Expressions

# Chapter 11: Select Operations

In [ ]:
// In programs where exceptions must be avoided, use nothrow versions of new and delete. 
void f(int n)
{
    int∗ p = new(nothrow) int[n]; // allocate n ints on the free store
    if (p==nullptr) {// no memory available
        // ... handle allocation error ...
    }
    // ...
    operator delete(nothrow,p); // deallocate *p
}

In [21]:
int a[] {1, 2, 3, 4 ,5};
int v {8};
v += {88};
std::cout<<v;


96

In [ ]:
void algo(vector<int>& v)
{
    sort(v.begin(),v.end()); // sor t values
    // ...
    sort(v.begin(),v.end(),[](int x, int y) { return abs(x)<abs(y); }); // sor t absolute values
    // ...
}

* `[]`: no local names from the surrounding context can be used.
* `[&]`: all local names can be used, by reference.
* `[=]`: all local names can be used, by copy.
* `[&, capture-list]`: all local variables with names not mentioned in the list, by reference.
* `[=, capture-list]`: all local variables with names not mentioned in the list, by copy.


In [ ]:
void algo(vector<int>& v)
{
    int count = v.siz e();
    std::generate(v.begin(),v.end(),
        [count]()mutable{ return −−count; }
    );
}

In [ ]:
void f(string& s1, string& s2)
{
    function<void(char∗ b, char∗ e)> rev =
        [&](char∗ b, char∗ e) { if (1<e−b) { swap(∗b,∗−−e); rev(++b,e); } };
    rev(&s1[0],&s1[0]+s1.siz e());
    rev(&s2[0],&s2[0]+s2.siz e());
}

In [ ]:
// A lambda that captures nothing can be assigned to a pointer to function of an appropriate type. For
// example:
double (∗p1)(double) = [](double a) { return sqrt(a); };
double (∗p2)(double) = [&](double a) { return sqrt(a); }; // error : the lambda captures
double (∗p3)(int) = [](int a) { return sqrt(a); }; // error : argument types do not match

Conversions:
* `const_cast`: getting write access to something declared const.
* `static_cast`: reversing a well-defined implicit conversion.
* `reinterpret_cast`: changing the meaning of bit patterns.
* `dynamic_cast`: for dynamically checked class hierarchy navigation.

# Chapter 12: Functions

In [ ]:
#include<vector>
using namespace std;
template<class T, class U>
auto product(const vector<T>& x, const vector<U>& y) -> decltype(x∗y);

In [ ]:
constexpr int fac(int n)
{
    return (n>1) ? n∗fac(n−1) : 1;
}

constexpr int f9 = fac(9); // must be evaluated at compile time
int f5 = fac(5); // may be evaluated at compile time
int fn = fac(n); // evaluated at run time (n is a var iable)
constexpr int f6 = fac(6); // must be evaluated at compile time
constexpr int fnn = fac(n); // error : can’t guarantee compile-time evaluation (n is a var iable)


In [ ]:
[[noreturn]] void exit(int); // exit will never retur n

In [ ]:
void f(int a)
{
    while (a−−) {
        static int n = 0; // initialized once
        int x = 0; // initialized ’a’ times in each call of f()
        cout << "n == " << n++ << ", x == " << x++ << '\n';
    }
}

In [ ]:
int f(int, int =0, char∗ =nullptr); // OK
int g(int =0, int =0, char∗); // error
int h(int =0, int, char∗ =nullptr); // error

In [ ]:
void (∗f1)(string) = &error; // OK: same as = error
void (∗f2)(string) = error; // OK: same as = &error
void g()
{
    f1("Vasa"); // OK: same as (*f1)("Vasa")
    (∗f1)("Mary Rose"); // OK: as f1("Mary Rose")
}

# Chapter 13: Exception Handling

In [ ]:

void f(int i)
{
    int∗ p = new int[10];
    // ...
    if (i<0) {
        delete[] p; // delete before the throw or leak
        throw Bad();
    }
    // ...
}

In [ ]:
double compute(double) noexcept; // may not throw an exception

template<typename T>
void my_fct(T& x) noexcept(Is_pod<T>());

# Chapter 14: Namespaces

In [ ]:
* `::f()`: global scope
* Classes are namespaces

In [ ]:
// use namespace alias to shorten names:
namespace ATT = American_Telephone_and_Telegraph;

# Chapter 15: Source Files and Programs

In [ ]:
static int x1 = 1; // internal linkage: not accessible from other translation units
const char x2 = 'a'; // internal linkage: not accessible from other translation units

int x1 = 1; // exter nal linkage: accessible from other translation units
extern const char x2 = 'a'; // exter nal linkage: accessible from other translation units

# Chapter 16: Classes

In [ ]:
class Date {
    int d, m, y;
public:
    explicit Date(int dd =0, int mm =0, int yy =0);
    // ...
};
Date d1 {15}; // OK: considered explicit
Date d2 = Date{15}; // OK: explicit
Date d3 = {15}; // error : = initialization does not do implicit conversions
Date d4 = 15; // error : = initialization does not do implicit conversions

In [ ]:
// const indicates that these functions do not modify the state of a Date.
class Date {
    int d, m, y;
public:
    int day() const { return d; }
    int month() const { return m; }
    int year() const;
    void add_year(int n); // add n years
    // ...
};

In [ ]:
// mutable, meaning that it can be modified even in a const
object
class Date {
public:
    // ...
    string string_rep() const; // string representation
    private:
    mutable bool cache_valid;
    mutable string cache;
    void compute_cache_value() const; // fill (mutable) cache
    // ...
};
string Date::string_rep() const
{
    if (!cache_valid) {
        compute_cache_value();
        cache_valid = true;
    }
    return cache;
}

void f(Date d, const Date cd)
{
    string s1 = d.string_rep();
    string s2 = cd.string_rep(); // OK!
    // ...
}

In [ ]:
// const does not apply (transitively) to objects accessed through pointers or references.

struct cache {
    bool valid;
    string rep;
};
class Date {
public:
    // ...
    string string_rep() const; // string representation
    private:
    cache∗ c; // initialize in constr uctor
    void compute_cache_value() const; // fill what cache refers to
    // ...
};
string Date::string_rep() const
{
    if (!c−>valid) {
        compute_cache_value();
        c−>valid = true;
    }
    return c−>rep;
}

# Chapter 17: Construction, Cleanup, Copy, and Move

In [ ]:
class Node {
    // ...
    static int node_count; // declaration
};
int Node::node_count = 0; // definition

Copy for a class X is defined by two operations:
* Copy constructor: X(const X&)
* Copy assignment: X& operator=(const X&)

In [ ]:
template<class T>
void swap(T& a, T& b) // "perfect swap" (almost)
{
    T tmp = std::move(a);
    a = std::move(b);
    b = std::move(tmp);
}

In [ ]:
class Base {
    // ...
    Base& operator=(const Base&) = delete;// disallow copying
    Base(const Base&) = delete;
    Base& operator=(Base&&) = delete; // disallow moving
    Base(Base&&) = delete;
    };
Base x1;
Base x2 {x1}; // error : no copy constr uctor

# Chapter 18: Operator Overloading

The following operators cannot be defined by a user:
* `::`: scope resolution
* `.`:  member selection
* `.∗`: member selection through pointer to member
* `?:` conditional evaluation

# Chapter 19: Special Operators

`[] () −> ++ −− new delete`

In [ ]:
template<typename T>
class Ptr {
    T∗ ptr;
    T∗ array;
    int sz;
public:
    template<int N>
    Ptr(T∗ p, T(&a)[N]); // bind to array a, sz==N, initial value p
    Ptr(T∗ p, T∗ a, int s); // bind to array a of size s, initial value p
    Ptr(T∗ p); // bind to single object, sz==0, initial value p
    Ptr& operator++(); // prefix
    Ptr operator++(int); // postfix
    Ptr& operator−−(); // prefix
    Ptr operator−−(int); // postfix
    T& operator∗(); // prefix
};

In [ ]:
constexpr rc_max {4}; // row and column size
class Matrix;
class Vector {
    float v[rc_max];
    // ...
    friend Vector operator∗(const Matrix&, const Vector&);
};

class Matrix {
    Vector v[rc_max];
    // ...
    friend Vector operator∗(const Matrix&, const Vector&);
};

Vector operator∗(const Matrix& m, const Vector& v)
{
    Vector r;
    for (int i = 0; i!=rc_max; i++) { // r[i] = m[i] * v;
        r.v[i] = 0;
        for (int j = 0; j!=rc_max; j++)
            r.v[i] += m.v[i].v[j] ∗ v.v[j];
    }
    return r;
}

# Chapter 20: Derived Classes

To get runtime polymorphic behavior in C++, the member functions called must be virtual and objects must be manipulated through
pointers or references. When manipulating an object directly (rather than through a pointer or reference), its exact type is known by the compiler so that run-time polymorphism is not needed

* `virtual`: The function may be overridden
* `=0`: The function must be virtual and must be overridden
* `override`: The function is meant to override a virtual function in a base class
* `final`: The function is not meant to be overridden 

In [56]:
#include<iostream>
using namespace std;

struct Base {
    void ff(int x) {
        std::cout<<x+1;
    }
};

struct Derived: Base {
    void fgf(int x) {
        std::cout<<x;
    }
};

In [57]:
Derived b{};
b.ff(1);

2

In [ ]:
template<class T>
struct Vector : std::vector<T> {
    using vector<T>::vector; // inherit constr uctors
    T& operator=[](size_type i) { check(i); return this−>elem(i); }
    const T& operator=(size_type i) const { check(i); return this−>elem(i); }
    void check(siz e_type i) { if (this−>size()<i) throw Bad_index(i); }
};
Vector<int> v { 1, 2, 3, 5, 8 }; // OK: use initializer-list constructor from std::vector

In [ ]:
// A class with one or more pure virtual functions is an abstract class, and no objects of that abstract
// class can be created:
class Shape { // abstract class
public:
    virtual void rotate(int) = 0; // pure virtual function
    virtual void draw() const = 0; // pure virtual function
    virtual bool is_closed() const = 0; // pure virtual function
    // ...
    virtual ˜Shape(); // vir tual
};

* If it is private, its name can be used only by member functions and friends of the class in which it is declared
* If it is protected, its name can be used only by member functions and friends of the class in which it is declared and by member functions and friends of classes derived from this class

* If it is public, its name can be used by any function

* public derivation makes the derived class a subtype of its base. For example, X is a kind of B. This is the most common form of derivation.
* private bases are most useful when defining a class by restricting the interface to a base so that stronger guarantees can be provided. For example, B is an implementation detail of Z. The Vector of pointers template that adds type checking to its Vector<void∗> base from §25.3
is a good example.
* protected bases are useful in class hierarchies in which further derivation is the norm. Like private derivation, protected derivation is used to represent implementation details. The Ival_slider from §21.2.2 is a good example.

* If B is a private base, its public and protected members can be used only by member functions and friends of D. Only friends and members of D can convert a D∗ to a B∗.
* If B is a protected base, its public and protected members can be used only by member functions and friends of D and by member functions and friends of classes derived from D. Only friends and members of D and friends and members of classes derived from D can convert a D∗ to a B∗.
* If B is a public base, its public members can be used by any function. In addition, its protected members can be used by members and friends of D and members and friends of classes derived from D. Any function can convert a D∗ to a B∗.

In [ ]:
using Pstd_mem = void (Std_interface::∗)(); // pointer-to-member type
void f(Std_interface∗ p)
{
    Pstd_mem s = &Std_interface::suspend; // pointer to suspend()
    p−>suspend(); // direct call
    p−>∗s(); // call through pointer to member
}

# Chapter 21: Class Hierarchies

# Chapter 22: Run-Time Type Information

In [ ]:
// A dynamic_cast to void∗ can be used to determine the address of the beginning of an object of polymorphic type. For example:
void g(Ival_box∗ pb, Date∗ pd)
{
    void∗ pb2 = dynamic_cast<void∗>(pb); // OK
    void∗ pd2 = dynamic_cast<void∗>(pd); // error : Date not polymorphic
}

In [ ]:
void f(Shape& r, Shape∗ p)
{
    typeid(r); // type of the object referred to by r
    typeid(∗p); // type of the object pointed to by p
    typeid(p); // type of the pointer, that is, Shape* (uncommon, except as a mistake)
}

# Chapter 23: Templates

In [ ]:
template<typename Scalar>
class complex {
    Scalar re, im;
public:
    complex() :re{}, im{} {} // default constructor
    template<typename T>
    complex(T rr, T ii =0) :re{rr}, im{ii} { }
    complex(const complex&) = default; // copy constr uctor
    template<typename T>
    complex(const complex<T>& c) : re{c.real()}, im{c.imag()} { }
    // ...
};

In [ ]:
// A member template cannot be virtual. For example:
class Shape {
// ...
template<typename T>
virtual bool intersect(const T&) const =0; // error : vir tual template
};

In [ ]:
template<typename T, typename Compare = std::less<T>>
void sort(vector<T>& v) // definition
// Shell sort (Knuth, Vol. 3, pg. 84)
{
    Compare cmp; // make a default Compare object
    const size_t n = v.siz e();
    for (int gap=n/2; 0<gap; gap/=2)
        for (int i=gap; i<n; i++)
            for (int j=i−gap; 0<=j; j−=gap)
                if (cmp(v[j+gap],v[j]))
                    swap(v[j],v[j+gap]);
}

In [ ]:
template<typename T, int max>
struct Buffer {
    T buf[max];
public:
    // ...
};
template<typename T, int max>
T& lookup(Buffer<T,max>& b, const char∗ p);
Record& f(Buffer<string,128>& buf, const char∗ p)
{
    return lookup(buf,p); // use the lookup() where T is string and i is 128
}

In [ ]:
template<typename T, typename Allocator = allocator<T>> vector;
using Cvec = vector<char>; // both arguments are bound
Cvec vc = {'a', 'b', 'c'}; // vc is a vector<char,allocator<char>>
template<typename T>
using Vec = vector<T,My_alloc<T>>; // vector using my allocator (2nd argument is bound)
Vec<int> fib = {0, 1, 1, 2, 3, 5, 8, 13}; // fib is a vector<int,My_alloc<int>>

# Chapter 24: Generic Programming